In [4]:
# %%
import sys
sys.path.append('..')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf

from erank.utils import load_directions_matrix_from_task_sweep
        


In [5]:
run_path = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/f_mnist-same_init-taskidxXXX_230622_162800/outputs/f_mnist-same_init-taskidx0-seed0_230622_162819'

if isinstance(run_path, str):
    run_path = Path(run_path)
# load config
loaded_config = OmegaConf.load(run_path/'.hydra'/'config.yaml')
config = loaded_config.config
# get model class
model_name = config.model.name
model_class = get_model_class(model_name)
# get best epoch
best_epoch_file = run_path / 'best_epoch.txt'
assert best_epoch_file.exists(), 'No best epoch file found.'

In [6]:
with best_epoch_file.open() as f:
    t = f.read()
    print(t)

12


In [7]:
run_path.stem

'f_mnist-same_init-taskidx0-seed0_230622_162819'

In [8]:
p = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/f_mnist-same_init-taskidxXXX_230622_162800/outputs'
dir_matrix = load_directions_matrix_from_task_sweep(p)

Loading /system/user/beck/pwbeck/projects/regularization/erank/outputs/f_mnist-same_init-taskidxXXX_230622_162800/outputs/f_mnist-same_init-taskidx4-seed0_230622_162834: 100%|██████████| 8/8 [00:03<00:00,  2.04it/s]


In [9]:
dir_matrix.shape, dir_matrix.device

(torch.Size([8, 669706]), device(type='cuda', index=0))